In [7]:
import pyarrow as pa

In [2]:
batch = pa.RecordBatch.from_arrays([range(1,1_000_000),
                                    range(1,1_000_000),
                                    range(1,1_000_000)],
                                   names=["x", "y", "z"])
batch.nbytes / 1024**2

22.888160705566406

In [3]:
with pa.ipc.new_file("test.arrow", schema=batch.schema) as f:
    for i in range(50):
        f.write_batch(batch)

In [4]:
!ls -lah

total 1.2G
drwxr-xr-x 3 root root 4.0K Sep 29 18:56 .
drwxr-xr-x 1 root root 4.0K Sep 29 18:55 ..
drwxr-xr-x 2 root root 4.0K Sep 29 18:55 .ipynb_checkpoints
-rw-r--r-- 1 root root   72 Sep 29 18:55 demo.ipynb
-rw-r--r-- 1 root root 1.2G Sep 29 18:56 test.arrow


In [13]:
# not enough memory!
#with pa.ipc.open_file("test.arrow") as f:
#    tbl = f.read_all()

In [3]:
import mmap
with open("test.arrow", "rb") as f:
    mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)

In [5]:
mm[:10]

b'ARROW1\x00\x00\xff\xff'

In [8]:
with pa.ipc.open_file(mm) as f:
    tbl = f.read_all()

In [10]:
import pyarrow.compute as pc
pc.sum(tbl["x"])

<pyarrow.Int64Scalar: 24999975000000>

In [11]:
pc.sum(tbl["y"])

<pyarrow.Int64Scalar: 24999975000000>

In [12]:
pc.sum(tbl["z"])

<pyarrow.Int64Scalar: 24999975000000>